In [138]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row, column, gridplot
from bokeh.models import (
    ColumnDataSource, CDSView, GroupFilter, 
    Div, BooleanFilter, DataRange1d, Plot
)
from bokeh.palettes import Category10
import itertools
from bokeh.plotting import reset_output
from bokeh.io import export_png

# Load tables and view them

In [2]:
# load medium growth tests df
df1 = pd.read_csv('../data/medium_growths.csv', sep = ",", index_col=0)

# clean up dirty columns
df1["species"] = df1["species"].map(lambda x: x.strip('.jpg'))
df1["concentration"] = df1["concentration"].replace(np.nan, "NA")
df1['date'] = pd.to_datetime(df1['date'])

df1.head(10)

,area,concentration,date,dupl,medium,species
0,82447.957601,47.5,2018-06-22,1.0,MEA,S.commune
1,7853.981634,47.5,2018-06-22,1.0,MEA,S.commune
2,8494.866535,47.5,2018-06-22,1.0,MEA,S.commune
3,13684.777599,47.5,2018-06-22,2.0,MEA,S.commune
4,34636.059006,47.5,2018-06-22,2.0,MEA,S.commune
5,7542.963961,47.5,2018-06-22,2.0,MEA,S.commune
6,4536.459792,47.5,2018-06-22,1.0,MEA,S.commune.DSC3
7,4778.362426,47.5,2018-06-22,1.0,MEA,S.commune.DSC3
8,6082.123377,47.5,2018-06-22,1.0,MEA,S.commune.DSC3
9,6647.610055,47.5,2018-06-22,2.0,MEA,S.commune.DSC3


In [3]:
# load medium growth tests df
df2 = pd.read_csv('../data/tempLight_growths.csv', sep = ",", index_col=0)

# clean up dirty columns
df2["species"] = df2["species"].map(lambda x: x.strip('.jpg'))
df2['date'] = pd.to_datetime(df2['date'])

df2.head(10)

,area,date,dupl,light_source,species,temperature
0,205887.416146,2018-07-03,1,dark,A.oryzae,28
1,205887.416146,2018-07-03,1,dark,A.oryzae,28
2,205887.416146,2018-07-03,1,dark,A.oryzae,28
3,205887.416146,2018-07-03,1,dark,A.oryzae,28
4,205887.416146,2018-07-03,1,dark,A.oryzae,28
5,205887.416146,2018-07-03,1,dark,A.oryzae,28
6,205887.416146,2018-07-03,1,dark,A.oryzae,30
7,205887.416146,2018-07-03,1,dark,A.oryzae,30
8,205887.416146,2018-07-03,1,dark,A.oryzae,30
9,205887.416146,2018-07-03,1,light,P.ostreatus,25


# Calculate average growth per day per plate 
(plate = media?)

In [4]:
allSpecies = list(set(df1["species"]))
allSpecies.pop(allSpecies.index("Nc"))
print("Species:", allSpecies)

allMedia = list(set(df1["medium"]))
print("Media tested:", allMedia)

outputList = []

for specie in allSpecies:
    specieDf = df1.loc[df1["species"] == specie]
    
    for medium in allMedia:
        mediumDf = specieDf.loc[specieDf["medium"] == medium]
        
        allConcentrations = list(set(mediumDf["concentration"]))
        
        for conc in allConcentrations:
            concDf = mediumDf.loc[mediumDf["concentration"] == conc]
            
            allDates = list(set(concDf["date"]))
            
            for date in allDates:
                dateDf = concDf.loc[mediumDf["date"] == date]

                dayNo = sorted(allDates).index(date) + 1

#                 for dupl in set(dateDf["dupl"]):
#                     duplDf = dateDf.loc[dateDf["dupl"] == dupl]
#                 print(specie, medium, conc, date, dayNo, dateDf["area"].mean())
                
                outputList.append({
                    "specie": specie, 
                    "medium": medium, 
                    "concentration": conc,
                    "date": date, 
                    "day": dayNo,
                    "average area": dateDf["area"].mean()
                })

averageGrowth = pd.DataFrame(outputList)
averageGrowth.head(15)

Species: ['S.commune', 'A.oryzae', 'S.commune.DSC3', 'P.ostreatus']
Media tested: ['MEA', 'PDA', 'YPD']


,average area,concentration,date,day,medium,specie
0,810890.616985,50,2018-06-26,5,MEA,S.commune
1,345036.408755,50,2018-06-24,3,MEA,S.commune
2,29391.170071,50,2018-06-22,1,MEA,S.commune
3,564641.065624,50,2018-06-25,4,MEA,S.commune
4,130796.544941,50,2018-06-23,2,MEA,S.commune
5,515163.599323,52.5,2018-06-26,5,MEA,S.commune
6,192570.728486,52.5,2018-06-24,3,MEA,S.commune
7,20214.054331,52.5,2018-06-22,1,MEA,S.commune
8,375491.007939,52.5,2018-06-25,4,MEA,S.commune
9,56030.304977,52.5,2018-06-23,2,MEA,S.commune


In [131]:
MEA_df = averageGrowth.loc[averageGrowth["medium"] == "MEA"]
MEA_df.loc[:,("concentration")] = MEA_df["concentration"].astype(str)

YPD_df = averageGrowth.loc[averageGrowth["medium"] == "YPD"]
YPD_df.loc[:,("concentration")] = YPD_df["concentration"].astype(str)

PDA_df = averageGrowth.loc[averageGrowth["medium"] == "PDA"]
PDA_df.loc[:, ("concentration")] = PDA_df["concentration"].astype(str)

reset_output()
output_file("../data/growth_media.html", title="Media variations growth")

# A. oryzae plots
p1 = figure(title="A.oryzae MEA", 
            plot_width=500, plot_height=250)
p1.xaxis.axis_label = "Day"
p1.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(MEA_df.loc[MEA_df["specie"] == "A.oryzae"])
for conc, c in zip(set(MEA_df.loc[MEA_df["specie"] == "A.oryzae"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p1.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p1.legend.location = "top_left"
p1.legend.click_policy="hide"

p2 = figure(title="A.oryzae YPD", 
            plot_width=500, plot_height=250)
p2.xaxis.axis_label = "Day"
p2.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(YPD_df.loc[YPD_df["specie"] == "A.oryzae"])
for conc, c in zip(set(YPD_df.loc[YPD_df["specie"] == "A.oryzae"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p2.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p2.legend.location = "top_left"
p2.legend.click_policy="hide"

p3 = figure(title="A.oryzae PDA", 
            plot_width=500, plot_height=250)
p3.xaxis.axis_label = "Day"
p3.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(PDA_df.loc[PDA_df["specie"] == "A.oryzae"])
for conc, c in zip(set(PDA_df.loc[PDA_df["specie"] == "A.oryzae"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p3.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p3.legend.location = "top_left"
p3.legend.click_policy="hide"

# P. ostreatus plots
p4 = figure(title="P.ostreatus MEA", 
            plot_width=500, plot_height=250)
p4.xaxis.axis_label = "Day"
p4.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(MEA_df.loc[MEA_df["specie"] == "P.ostreatus"])
for conc, c in zip(set(MEA_df.loc[MEA_df["specie"] == "P.ostreatus"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])

           ]
          )
    p4.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p4.legend.location = "top_left"
p4.legend.click_policy="hide"

p5 = figure(title="P.ostreatus YPD", 
            plot_width=500, plot_height=250)
p5.xaxis.axis_label = "Day"
p5.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(YPD_df.loc[YPD_df["specie"] == "P.ostreatus"])
for conc, c in zip(set(YPD_df.loc[YPD_df["specie"] == "P.ostreatus"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p5.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p5.legend.location = "top_left"
p5.legend.click_policy="hide"

p6 = figure(title="P.ostreatus PDA", 
            plot_width=500, plot_height=250)
p6.xaxis.axis_label = "Day"
p6.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(PDA_df.loc[PDA_df["specie"] == "P.ostreatus"])
for conc, c in zip(set(PDA_df.loc[PDA_df["specie"] == "P.ostreatus"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p6.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p6.legend.location = "top_left"
p6.legend.click_policy="hide"

# S. commune plots
p7 = figure(title="S.commune MEA", 
            plot_width=500, plot_height=250)
p7.xaxis.axis_label = "Day"
p7.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(MEA_df.loc[MEA_df["specie"] == "S.commune"])
for conc, c in zip(set(MEA_df.loc[MEA_df["specie"] == "S.commune"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p7.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p7.legend.location = "top_left"
p7.legend.click_policy="hide"

p8 = figure(title="S.commune YPD", 
            plot_width=500, plot_height=250)
p8.xaxis.axis_label = "Day"
p8.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(YPD_df.loc[YPD_df["specie"] == "S.commune"])
for conc, c in zip(set(YPD_df.loc[YPD_df["specie"] == "S.commune"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p8.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p8.legend.location = "top_left"
p8.legend.click_policy="hide"

p9 = figure(title="S.commune PDA", 
            plot_width=500, plot_height=250)
p9.xaxis.axis_label = "Day"
p9.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(PDA_df.loc[PDA_df["specie"] == "S.commune"])
for conc, c in zip(set(PDA_df.loc[PDA_df["specie"] == "S.commune"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])

           ]
          )
    p9.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p9.legend.location = "top_left"
p9.legend.click_policy="hide"

# S. commune.DSC3 plots
p10 = figure(title="S.commune.DSC3 MEA", 
            plot_width=500, plot_height=250)
p10.xaxis.axis_label = "Day"
p10.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(MEA_df.loc[MEA_df["specie"] == "S.commune.DSC3"])

for conc, c in zip(set(MEA_df.loc[MEA_df["specie"] == "S.commune.DSC3"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p10.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p10.legend.location = "top_left"
p10.legend.click_policy="hide"

p11 = figure(title="S.commune.DSC3 YPD", 
            plot_width=500, plot_height=250)
p11.xaxis.axis_label = "Day"
p11.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(YPD_df.loc[YPD_df["specie"] == "S.commune.DSC3"])
for conc, c in zip(set(YPD_df.loc[YPD_df["specie"] == "S.commune.DSC3"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p11.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p11.legend.location = "top_left"
p11.legend.click_policy="hide"

p12 = figure(title="S.commune.DSC3", 
            plot_width=500, plot_height=250)
p12.xaxis.axis_label = "Day"
p12.yaxis.axis_label = "average area (pixels^2)"

source = ColumnDataSource(PDA_df.loc[PDA_df["specie"] == "S.commune.DSC3"])
for conc, c in zip(set(PDA_df.loc[PDA_df["specie"] == "S.commune.DSC3"]["concentration"]), Category10[3]):
    view = CDSView(source=source, 
           filters = [
               BooleanFilter([True if y_val == conc else False for y_val in source.data['concentration']])
           ]
          )
    p12.circle(x = "day", y = "average area", source=source, view=view, color = c, legend = conc)

p12.legend.location = "top_left"
p12.legend.click_policy="hide"

# output
grid = gridplot([[p1, p4, p7, p10], [p2, p5, p8, p11], [p3, p6, p9, p12]])

show(grid)

In [24]:
allSpecies = list(set(df2["species"]))
# allSpecies.pop(allSpecies.index("Nc"))
print("Species:", allSpecies)

allSources = list(set(df2["light_source"]))
print("Light sources tested:", allSources)

allTemps = list(set(df2["temperature"]))
print("Temperatures tested:", allTemps)

outputList = []

for specie in allSpecies:
    specieDf = df2.loc[df2["species"] == specie]
    
    for light in allSources:
        lightDf = specieDf.loc[specieDf["light_source"] == light]
        
        for temp in allTemps:
            tempDf = lightDf.loc[lightDf["temperature"] == temp]
            
            allDates = list(set(tempDf["date"]))
            
            for date in allDates:
                dateDf = tempDf.loc[tempDf["date"] == date]

                dayNo = allDates.index(date) + 1

                outputList.append({
                    "specie": specie, 
                    "light": light, 
                    "temperature": temp,
                    "date": date, 
                    "day": dayNo,
                    "average area": dateDf["area"].mean()
                })

averageGrowth2 = pd.DataFrame(outputList)
averageGrowth2.head(10)

Species: ['P.ostreatus', 'A.oryzae', 'S.commune.DSC3', 'S.commune']
Light sources tested: ['dark', 'light']
Temperatures tested: [25, 28, 37, 30]


,average area,date,day,light,specie,temperature
0,205887.416146,2018-07-03,1,dark,P.ostreatus,28
1,15041.945625,2018-07-05,2,dark,P.ostreatus,28
2,15041.945625,2018-07-04,3,dark,P.ostreatus,28
3,205887.416146,2018-07-03,1,light,P.ostreatus,25
4,15041.945625,2018-07-05,2,light,P.ostreatus,25
5,15041.945625,2018-07-04,3,light,P.ostreatus,25
6,205887.416146,2018-07-03,1,dark,A.oryzae,28
7,15041.945625,2018-07-05,2,dark,A.oryzae,28
8,15041.945625,2018-07-04,3,dark,A.oryzae,28
9,205887.416146,2018-07-03,1,dark,A.oryzae,30
